In [7]:
import pandas as pd
import re

def procesar_poblacion_urbana_rural(df, año_censo):
    """
    Procesa datos de población urbana y rural por distrito, estandarizando
    los nombres de las áreas para el censo de 2005.
    """
    datos_procesados = []
    area_actual = {"ubigeo": "", "distrito": ""}
    
    # Reiniciar índices y columnas
    df = df.reset_index(drop=True)
    df.columns = [str(i) for i in range(len(df.columns))]
    
    i = 0
    while i < len(df):
        fila = df.iloc[i]
        
        # 1. Buscar línea con "AREA #"
        if pd.notna(fila['0']) and 'AREA #' in str(fila['0']):
            area_texto = str(fila['0'])
            
            # Extraer ubigeo (6 dígitos)
            ubigeo_match = re.search(r'AREA #\s+(\d{6})', area_texto)
            if ubigeo_match:
                area_actual['ubigeo'] = ubigeo_match.group(1)
            
            # Extraer nombre del distrito
            distrito_match = re.search(r'\d{6}\s+(.+)', area_texto)
            if distrito_match:
                area_actual['distrito'] = distrito_match.group(1).strip()
            else:
                if len(df.columns) > 1 and pd.notna(fila['1']):
                    area_actual['distrito'] = str(fila['1']).strip()
            
            i += 1
            continue
        
        # 2. Buscar encabezado de tabla "Categorías"
        if (pd.notna(fila['0']) and 'Categorías' in str(fila['0']) and
            pd.notna(fila['1']) and 'Casos' in str(fila['1'])):
            
            # Procesar las 3 filas de datos
            for offset in [1, 2, 3]:
                if i + offset < len(df):
                    fila_datos = df.iloc[i + offset]
                    
                    if (pd.notna(fila_datos['0']) and 
                        any(x in str(fila_datos['0']) for x in ['Area Urbana', 'Area Rural', 'Total'])):
                        
                        try:
                            tipo_area_original = str(fila_datos['0']).strip()
                            
                            # CORREGIDO: Estandarizar el valor de tipo_area AQUI
                            if tipo_area_original == 'Area Urbana':
                                tipo_area = 'Urbano'
                            elif tipo_area_original == 'Area Rural':
                                tipo_area = 'Rural'
                            else:
                                tipo_area = 'Total'

                            casos = int(float(str(fila_datos['1']).replace(',', '').strip()))
                            porcentaje = float(str(fila_datos['2']).replace('%', '').strip()) if pd.notna(fila_datos['2']) and str(fila_datos['2']) != '' else None
                            acumulado = float(str(fila_datos['3']).replace('%', '').strip()) if pd.notna(fila_datos['3']) and str(fila_datos['3']) != '' else None
                            
                            datos_procesados.append({
                                'ubigeo': area_actual['ubigeo'],
                                'distrito': area_actual['distrito'],
                                'tipo_area': tipo_area, # Usar el valor estandarizado
                                'poblacion': casos,
                                'porcentaje': porcentaje,
                                'acumulado_porcentaje': acumulado,
                                'anio': año_censo
                            })
                        except (ValueError, TypeError) as e:
                            print(f"DEBUG - Error procesando fila {i + offset}: {e}")
                            continue
            
            i += 3 # Mueve el índice después de procesar la tabla
        
        i += 1
    
    # Crear DataFrame final
    df_final = pd.DataFrame(datos_procesados)
    
    if not df_final.empty:
        df_final['ubigeo'] = df_final['ubigeo'].astype(str).str.zfill(6)
        df_final['tipo_area'] = df_final['tipo_area'].astype('category')
        
    return df_final


ruta = r"D:\Mateo\ICSI\Proyecto\Data\Raw\Poblacion\Data\Inputs\Censo\censo_pob_area_2005.xlsx"
df = pd.read_excel(ruta)
print("=== PRIMERAS FILAS DEL ARCHIVO ORIGINAL ===")
print(df.head(15))

df_procesado = procesar_poblacion_urbana_rural(df, 2005)

print("\n=== RESULTADO FINAL ===")
print(df_procesado.info())
print("\n=== PRIMEROS REGISTROS ===")
print(df_procesado.head())
print(f"\n=== DISTRITOS ÚNICOS ===")
print(df_procesado['distrito'].unique())

ruta_destino = r"D:\Mateo\ICSI\Proyecto\Data\Raw\Poblacion\Data\Processing\pob_censo_area_2005.csv"

# CORREGIDO: Verificar si el DataFrame no está vacío antes de guardar
if not df_procesado.empty:
    df_procesado.to_csv(ruta_destino, encoding='latin1', index=False)
    print("\nCSV guardado correctamente.")
else:
    print("\nDataFrame vacío, no se guardó el CSV.")

=== PRIMERAS FILAS DEL ARCHIVO ORIGINAL ===
   Area Urbano / Rural   Unnamed: 1 Unnamed: 2   Unnamed: 3
0                  NaN          NaN        NaN          NaN
1        AREA # 010101  CHACHAPOYAS        NaN          NaN
2                  NaN          NaN        NaN          NaN
3           Categorías        Casos          %  Acumulado %
4          Area Urbana        21360      94.96        94.96
5           Area Rural         1133       5.04          100
6                Total        22493        100          100
7                  NaN          NaN        NaN          NaN
8        AREA # 010102     ASUNCION        NaN          NaN
9                  NaN          NaN        NaN          NaN
10          Categorías        Casos          %  Acumulado %
11         Area Urbana          157      44.48        44.48
12          Area Rural          196      55.52          100
13               Total          353        100          100
14                 NaN          NaN        NaN          